In [1]:
using CSV
using DataFrames

df = CSV.read("uwcourses_updated.csv", DataFrame)

list_courses = []
for col in eachcol(df)
    for course in col
        if ismissing(course) == true
            continue
        end
        
        push!(list_courses, course)
    end
end
courses = collect(Set(list_courses)) 

println(df)

91×3 DataFrame
 Row │ computer-science               data-science                   economics                         
     │ String31                       String31?                      String?                           
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ COMP SCI/MATH 240              MATH 221                       MATH 221
   2 │ COMP SCI/E C E 252             MATH 222                       MATH 171
   3 │ COMP SCI 300                   STAT 240                       MATH 217
   4 │ COMP SCI/E C E 354             STAT 340                       MATH 211
   5 │ COMP SCI 400                   COMP SCI 220                   MATH 213
   6 │ MATH 221                       COMP SCI 300                   MATH 211
   7 │ MATH 222                       COMP SCI 320                   ECON 205
   8 │ MATH 171                       L I S 461                      ECON 310
   9 │ MATH 217                       MATH 217   

In [2]:
# Load the CSV file into a DataFrame
courses_df = CSV.read("uwcourses_updated.csv", DataFrame)

# Convert all columns to a uniform type of Vector{String} to ensure compatibility
for col in names(courses_df)
    courses_df[!, col] = convert(Vector{String}, string.(courses_df[!, col]))
end

# Define a function to clean the input and extract the numerical part of the course name
function extract_course_number(course_name)
    if typeof(course_name) == String
        # Clean up the input by trimming whitespace
        cleaned_course_name = strip(course_name)
        # Match the first number in the string
        match_obj = match(r"\d+", cleaned_course_name)
        return match_obj !== nothing ? parse(Int, match_obj.match) : missing
    else
        return missing
    end
end

# Create a new DataFrame to store the extracted course numbers
course_numbers_df = DataFrame()

# Iterate over each column and extract the course numbers
for col in names(courses_df)
    # Extract course numbers for each value in the current column
    extracted_numbers = [extract_course_number(course_name) for course_name in courses_df[!, col]]
    # Add the extracted numbers as a new column in the new DataFrame
    course_numbers_df[!, col] = extracted_numbers
end

println(course_numbers_df)

91×3 DataFrame
 Row │ computer-science  data-science  economics 
     │ Int64             Int64?        Int64?    
─────┼───────────────────────────────────────────
   1 │              240           221        221
   2 │              252           222        171
   3 │              300           240        217
   4 │              354           340        211
   5 │              400           220        213
   6 │              221           300        211
   7 │              222           320        205
   8 │              171           461        310
   9 │              217           217        400
  10 │              222           570        410
  11 │              320           532        309
  12 │              340           539        431
  13 │              341           540        311
  14 │              375           656        324
  15 │              309           521        340
  16 │              311           535        101
  17 │              324           361        102
  

In [3]:
difficulty = Dict()

# Assuming both DataFrames have the same structure, iterate through them
for i in 1:nrow(courses_df)
    for col in names(courses_df)
        course_name = courses_df[i, col]
        difficulty_score = course_numbers_df[i, col]
        if !ismissing(course_name) && !ismissing(difficulty_score)
            difficulty[string(course_name)] = difficulty_score
        end
    end
end

println(difficulty)

Dict{Any, Any}("ECON 522" => 522, "GEOG 574" => 574, "STAT 433" => 433, "COMP SCI/MATH 240" => 240, "SOIL SCI 585" => 585, "ECON 409" => 409, "STAT 312" => 312, "ECON 570" => 570, "COMP SCI 412" => 412, "COMP SCI/E C E 252" => 252, "ECON/FINANCE 320" => 320, "ECON 468" => 468, "COMP SCI/B M I 576" => 576, "COMP SCI 541" => 541, "ECON 455" => 455, "GEOG 378" => 378, "GEN BUS 656" => 656, "ECON/HIST SCI 305" => 305, "ECON/AAE 371" => 371, "MATH 171" => 171, "L I S 501" => 501, "COMP SCI/E C E/ISYE 524" => 524, "COMP SCI/E C E 552" => 552, "COMP SCI/DS 579" => 579, "ECON 461" => 461, "MATH/STAT 309" => 309, "ECON/AAE/ENVIR ST 343" => 343, "ECON 460" => 460, "ECON 311" => 311, "STAT 453" => 453, "ECON 442" => 442, "STAT 240" => 240, "L I S 464" => 464, "MATH 535" => 535, "ECON 111" => 111, "COMP SCI/E C E 506" => 506, "I SY E 412" => 412, "COMP SCI 566" => 566, "L I S 407" => 407, "ECON/REAL EST/URB R PL 420" => 420, "COMP SCI/I SY E/M E 558" => 558, "ECON 302" => 302, "COMP SCI 220" => 22

In [4]:
credits_df = CSV.read("CS524-optimization-courses - credit-courses.csv", DataFrame)
credits = Dict()

for i in 1:nrow(courses_df)
    for col in names(courses_df)
        course_name = courses_df[i, col]
        credit = credits_df[i, col]
        if !ismissing(course_name) && !ismissing(credits_df)
            credits[string(course_name)] = credit
        end
    end
end

credits

Dict{Any, Any} with 190 entries:
  "ECON 522"           => 4
  "GEOG 574"           => 4
  "STAT 433"           => 3
  "COMP SCI/MATH 240"  => 3
  "SOIL SCI 585"       => 3
  "ECON 409"           => 4
  "STAT 312"           => 3
  "ECON 570"           => 4
  "COMP SCI 412"       => 3
  "COMP SCI/E C E 252" => 3
  "ECON/FINANCE 320"   => 3
  "ECON 468"           => 4
  "COMP SCI/B M I 576" => 3
  "COMP SCI 541"       => 3
  "ECON 455"           => 4
  "GEOG 378"           => 4
  "GEN BUS 656"        => 3
  "ECON/HIST SCI 305"  => 4
  "ECON/AAE 371"       => 3
  ⋮                    => ⋮

In [9]:
function group_courses(course_list, difficulty)
    new_dict_difficulty = Dict{String, Int}()
    
    for course in course_list
        for (key, value) in difficulty
            if course == key
                new_dict_difficulty[course] = value
            end
        end
    end

    return new_dict_difficulty
end

function group_courses(course_list, difficulty)
    # Initialize an empty dictionary to store courses grouped by major
    grouped_courses = Dict{String, Vector{Int}}()

    # Iterate over each course in the course list
    for course in course_list
        for (course_name, course_number) in difficulty
            # Check if the course name matches
            if course == course_name
                # Extract the major from the course name (everything before the last space)
                parts = split(course_name)
                major = join(parts[1:end-1], " ")

                # Add the course number to the appropriate major in the dictionary
                if haskey(grouped_courses, major)
                    push!(grouped_courses[major], course_number)
                else
                    grouped_courses[major] = [course_number]
                end
            end
        end
    end
    
    # Sort the course numbers for each major for easier readability
    for (major, numbers) in grouped_courses
        grouped_courses[major] = sort(numbers)
    end

    return grouped_courses
end

function create_schedule(courses::Dict{String, Vector{Int}})
    # Flatten the input dictionary into a list of (subject, course) tuples
    sorted_courses = []
    for (subject, course_list) in courses
        append!(sorted_courses, [(subject, course) for course in course_list])
    end
    
    # Sort the courses by their level (e.g., 100 < 200 < 300 < 400 < 500)
    sorted_courses = sort(sorted_courses, by = x -> parse(Int, string(x[2])[1]), rev=false)
    
    # Create a dictionary to store the schedule for 8 semesters
    schedule = Dict("Semester $i" => [] for i in 1:8)
    
    # Dictionary to track the last semester a course from each subject was taken
    last_taken = Dict{String, Int}()
    
    # Iterate over the sorted list and distribute courses to the schedule
    for (subject, course) in sorted_courses
        # Find an available semester where this course can be added
        added = false
        for semester in 1:8
            # Check if the semester already has 3 courses
            if length(schedule["Semester $semester"]) < 3
                # Ensure that no course from the same subject is taken in the same semester
                if !(subject in [x[1] for x in schedule["Semester $semester"]])
                    push!(schedule["Semester $semester"], (subject, course))
                    last_taken[subject] = semester
                    added = true
                    break
                end
            end
        end
        
        # If a course was not added to any semester, print a message (debugging)
        if !added
            println("You may take this course anytime in your later semesters $subject $course")
        end
    end

    # Ensure the output is printed in the correct order
    return schedule
end

create_schedule (generic function with 1 method)

In [10]:
using JuMP, GLPK

# Define the model with the GLPK solver
model = Model(GLPK.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#All elective variables
@variable(model, e_cs_theory >= 0)
@variable(model, e_cs_software_hardware >= 0)
@variable(model, e_cs_applications >= 0)
@variable(model, e_ds_machine_learning >= 0)
@variable(model, e_ds_advanced_computing >= 0)
@variable(model, e_ds_statistical_modelling >= 0)
@variable(model, e_econ_two_courses >= 0)

#Computer Science Constraints
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 200"] + 
    course_taken["COMP SCI 400"] >= 6 
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1 + e_cs_theory
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2 + e_cs_software_hardware
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1 + e_cs_applications
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI/E C E/M E 539"] +  
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 639"] >= 2 - e_cs_theory - e_cs_software_hardware - e_cs_applications
)

#Data Science Constraints
@constraint(
    model,
    course_taken["MATH 221"] + course_taken["MATH 217"] >= 1
)

@constraint(
    model,
    course_taken["MATH 222"] >= 1
)
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1 + e_ds_machine_learning
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1 + e_ds_advanced_computing
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1 + e_ds_statistical_modelling
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2 - e_ds_machine_learning - e_ds_advanced_computing - e_ds_statistical_modelling
)

#Econ Constraints
@constraint(
    model,
    2*course_taken["MATH 221"] + 
    course_taken["MATH 211"] + 
    course_taken["ECON 205"] >= 2
)
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    course_taken["ECON 101"] + 
    course_taken["ECON 102"] + 
    2*course_taken["ECON 111"] >= 2
)
@constraint(
    model,
    course_taken["ECON 301"] + course_taken["ECON 302"]  >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2 + e_econ_two_courses
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2 - e_econ_two_courses
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

@objective(model, Min, sum(course_taken[course] for course in courses))

# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with lowest number of courses:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

#Print Schedule Result
result = group_courses(final_taken, difficulty)
schedule = create_schedule(result)
for i in 1:8
    semester_key = "Semester $i"
    if length(schedule[semester_key]) > 0
        println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
    else
        println("$semester_key: No courses scheduled")
    end
end

Optimal solution found. Courses to enroll with lowest number of courses:
ECON 400
COMP SCI/E C E 354
COMP SCI/MATH 240
COMP SCI 537
ECON 570
ECON 301
COMP SCI 412
COMP SCI/E C E 252
ECON 321
COMP SCI/DS/I SY E 518
COMP SCI/E C E/M E 532
STAT 240
MATH 222
ECON 111
ECON 315
ECON 302
COMP SCI 520
COMP SCI 220
E C E/I SY E 570
MATH 375
COMP SCI 564
COMP SCI 200
COMP SCI 320
COMP SCI 300
L I S 461
COMP SCI 400
MATH 221
STAT 340
You may take this course anytime in your later semesters ECON 570
You may take this course anytime in your later semesters COMP SCI 537
You may take this course anytime in your later semesters COMP SCI 564
You may take this course anytime in your later semesters COMP SCI/DS/I SY E 518
Semester 1: ("ECON", 111), ("MATH", 221), ("COMP SCI/E C E", 252)
Semester 2: ("MATH", 222), ("COMP SCI", 200), ("STAT", 240)
Semester 3: ("COMP SCI", 220), ("COMP SCI/MATH", 240), ("MATH", 375)
Semester 4: ("ECON", 301), ("COMP SCI/E C E", 354), ("COMP SCI", 300)
Semester 5: ("ECON", 3

In [11]:
using JuMP, HiGHS

# Define the model with the HiGHS solver
model = Model(HiGHS.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

#Computer Science Constraints
@constraint(
    model,
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI/MATH 240"] + 
    course_taken["COMP SCI/E C E 252"] + 
    course_taken["COMP SCI/E C E 354"] + 
    course_taken["COMP SCI 200"] + 
    course_taken["COMP SCI 400"] >= 6
)

@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)

@constraint(
    model,
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["MATH 331"] + 
    course_taken["STAT 333"] + 
    course_taken["STAT 340"] + 
    course_taken["STAT 371"] + 
    course_taken["STAT/MATH 431"] + 
    course_taken["MATH 531"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI 520"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] >= 2
)
@constraint(
    model,
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 407"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/E C E/MATH 435"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH/STAT 475"] + 
    course_taken["COMP SCI/E C E 506"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/DS/I SY E 518"] + 
    course_taken["COMP SCI 520"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/I SY E 526"] + 
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 534"] + 
    course_taken["COMP SCI 536"] + 
    course_taken["COMP SCI 537"] + 
    course_taken["COMP SCI 538"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["COMP SCI 541"] + 
    course_taken["COMP SCI 542"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI/E C E 552"] + 
    course_taken["COMP SCI/I SY E/M E 558"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/E C E 561"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI 566"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 570"] + 
    course_taken["COMP SCI 571"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["COMP SCI/DS 579"] + 
    course_taken["COMP SCI 620"] + 
    course_taken["COMP SCI/I SY E 635"] + 
    course_taken["COMP SCI 640"] + 
    course_taken["COMP SCI 642"] + 
    course_taken["COMP SCI 639"] >= 2
)

#Data Science Constraints
@constraint(
    model,
    course_taken["MATH 221"] + course_taken["MATH 217"] >= 1
)

@constraint(
    model,
    course_taken["MATH 222"] >= 1
)
@constraint(
    model,
    course_taken["STAT 240"] +
    course_taken["STAT 340"] +
    course_taken["COMP SCI 220"] + 
    course_taken["COMP SCI 300"] + 
    course_taken["COMP SCI 320"] +
    course_taken["L I S 461"] + 
    course_taken["E C E/I SY E 570"] >= 7
)
@constraint(
    model,
    course_taken["COMP SCI/E C E/M E 532"] + 
    course_taken["COMP SCI/E C E/M E 539"] + 
    course_taken["COMP SCI 540"] + 
    course_taken["GEN BUS 656"] + 
    course_taken["I SY E 521"] + 
    course_taken["MATH 535"] + 
    course_taken["PHYSICS 361"] + 
    course_taken["STAT 451"] + 
    course_taken["STAT 453"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI 400"] + 
    course_taken["COMP SCI 412"] + 
    course_taken["COMP SCI/STAT 471"] + 
    course_taken["COMP SCI/MATH 513"] + 
    course_taken["COMP SCI/MATH 514"] + 
    course_taken["COMP SCI/E C E/I SY E 524"] + 
    course_taken["COMP SCI 544"] + 
    course_taken["COMP SCI 564"] + 
    course_taken["COMP SCI 565"] + 
    course_taken["COMP SCI/BMI 576"] + 
    course_taken["GEOG 573"] + 
    course_taken["GEOG 574"] + 
    course_taken["MATH 444"] >= 1
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 460"] + 
    course_taken["GEOG 579"] + 
    course_taken["I SY E 575"] + 
    course_taken["STAT/MATH 309"] + 
    course_taken["STAT 311"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT/MATH 310"] + 
    course_taken["STAT 312"] + 
    course_taken["STAT 349"] + 
    course_taken["STAT 351"] + 
    course_taken["STAT 421"] + 
    course_taken["STAT/M E 424"] + 
    course_taken["STAT 436"] + 
    course_taken["STAT 443"] + 
    course_taken["STAT 456"] + 
    course_taken["STAT 461"] + 
    course_taken["STAT 575"] + 
    course_taken["MATH 531"] + 
    course_taken["MATH/I SY E/OTM/STAT 632"] + 
    course_taken["MATH 635"] >= 1
)
@constraint(
    model,
    course_taken["MATH 320"] + 
    course_taken["MATH 340"] + 
    course_taken["MATH 341"] + 
    course_taken["MATH 375"] >= 1
)
@constraint(
    model,
    course_taken["COMP SCI/I SY E/MATH 425"] + 
    course_taken["COMP SCI/I SY E/MATH/STAT 525"] + 
    course_taken["COMP SCI/E C E 533"] + 
    course_taken["COMP SCI 559"] + 
    course_taken["COMP SCI/BMI 567"] + 
    course_taken["COMP SCI 577"] + 
    course_taken["E C E 203"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 695"] + 
    course_taken["GEOG 378"] + 
    course_taken["GEOG 572"] + 
    course_taken["GEOG 575"] + 
    course_taken["I SY E 323"] + 
    course_taken["I SY E 412"] + 
    course_taken["I SY E/M E 512"] + 
    course_taken["I SY E 612"] + 
    course_taken["INFO SYS 322"] + 
    course_taken["L I S 407"] + 
    course_taken["L I S 440"] + 
    course_taken["L I S 464"] + 
    course_taken["L I S 501"] + 
    course_taken["LSC 460"] + 
    course_taken["LSC 660"] + 
    course_taken["MATH 331"] + 
    course_taken["SOC 351"] + 
    course_taken["SOC/C&E SOC 618"] + 
    course_taken["SOC/C&E SOC 693"] + 
    course_taken["SOIL SCI 585"] + 
    course_taken["STAT 405"] + 
    course_taken["STAT 433"] >= 2
)

#Econ Constraints
@constraint(
    model,
    course_taken["MATH 221"] + 
    (course_taken["MATH 171"] + course_taken["MATH 217"]) + 
    (course_taken["MATH 211"] + course_taken["MATH 213"]) + 
    (course_taken["MATH 211"] + course_taken["ECON 205"]) >= 1
)
@constraint(
    model,
    course_taken["ECON 310"] + 
    course_taken["ECON 400"] + 
    course_taken["ECON 410"] + 
    course_taken["MATH/STAT 309"] + 
    course_taken["MATH/STAT 431"] + 
    course_taken["STAT 311"] + 
    course_taken["STAT 324"] + 
    course_taken["STAT 340"] >= 1
)
@constraint(
    model,
    (course_taken["ECON 101"] + course_taken["ECON 102"]) + 
    course_taken["ECON 111"] + course_taken["ECON 111"]>= 2
)
@constraint(
    model,
    (course_taken["ECON 301"] + course_taken["ECON 302"]) + 
    (course_taken["ECON 311"] + course_taken["ECON 312"]) >= 2
)
@constraint(
    model,
    course_taken["ECON 400"] + 
    course_taken["ECON 409"] + 
    course_taken["ECON 410"] + 
    course_taken["ECON 435"] + 
    course_taken["ECON 441"] + 
    course_taken["ECON 442"] + 
    course_taken["ECON 448"] + 
    course_taken["ECON 450"] + 
    course_taken["ECON 451"] + 
    course_taken["ECON 455"] + 
    course_taken["ECON 458"] + 
    course_taken["ECON 460"] + 
    course_taken["ECON 461"] + 
    course_taken["ECON 464"] + 
    course_taken["ECON 467"] + 
    course_taken["ECON 468"] + 
    course_taken["ECON 475"] + 
    course_taken["ECON/FINANCE 503"] + 
    course_taken["ECON 521"] + 
    course_taken["ECON 522"] + 
    course_taken["ECON/RMI 530"] + 
    course_taken["ECON/POP HLTH/PUB AFFR 548"] + 
    course_taken["ECON 570"] + 
    course_taken["ECON 580"] + 
    course_taken["ECON 621"] + 
    course_taken["ECON 623"] + 
    course_taken["ECON 661"] + 
    course_taken["ECON 664"] + 
    course_taken["ECON 666"] + 
    course_taken["ECON 690"] + 
    course_taken["ECON 695"] >= 2
)
@constraint(
    model,
    course_taken["ECON/FINANCE 300"] + 
    course_taken["ECON/HIST SCI 305"] + 
    course_taken["ECON/AAE/REAL EST/URB R PL 306"] + 
    course_taken["ECON 309"] + 
    course_taken["ECON 315"] + 
    course_taken["ECON/FINANCE 320"] + 
    course_taken["ECON 321"] + 
    course_taken["ECON 330"] + 
    course_taken["ECON/AAE/ENVIR ST 343"] + 
    course_taken["ECON 355"] + 
    course_taken["ECON 370"] + 
    course_taken["ECON/AAE 371"] + 
    course_taken["ECON 390"] + 
    course_taken["ECON/REAL EST/URB R PL 420"] + 
    course_taken["ECON/AAE 421"] + 
    course_taken["ECON/ENVIR ST/POLI SCI/URB R PL 449"] + 
    course_taken["ECON/AAE/INTL BUS 462"] + 
    course_taken["ECON 465"] + 
    course_taken["ECON/HISTORY 466"] + 
    course_taken["ECON/AAE 473"] + 
    course_taken["ECON/AAE 474"] + 
    course_taken["ECON/AAE 477"] + 
    course_taken["ECON/PHILOS 524"] + 
    course_taken["ECON/AAE 526"] + 
    course_taken["ECON/AAE/F&W ECOL 531"] + 
    course_taken["ECON/SOC 663"] + 
    course_taken["ECON/AAE/ENVIR ST/URB R PL 671"] >= 2
)

@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) >= 48
)

# Assume you take 18 credits every semester for 8 semesters while considering taking humanities, social sciecnes, language, and natural sciences with a total of 36 credits
@constraint(
    model,
    sum(credits[course] * course_taken[course] for course in courses) <= 108
)

# Maximize the difficulty and number CS courses while making the total number courses minimized
@objective(model, Max, sum(difficulty[course] * course_taken[course] for course in courses if occursin("COMP SCI", course)) - 0.5 * sum(course_taken[course] for course in courses))

set_optimizer_attribute(model, "presolve", "on")
# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll with most CS related:")
    final_taken = []
    for course in courses
        if value(course_taken[course]) > 0.5
            push!(final_taken, course)
            println(course)
        end
    end
else
    println("No optimal solution found.")
end

#Print Schedule Result
result = group_courses(final_taken, difficulty)
schedule = create_schedule(result)
for i in 1:8
    semester_key = "Semester $i"
    if length(schedule[semester_key]) > 0
        println("$semester_key: ", join(["$course" for course in schedule[semester_key]], ", "))
    else
        println("$semester_key: No courses scheduled")
    end
end


ArgumentError: ArgumentError: Package HiGHS not found in current path.
- Run `import Pkg; Pkg.add("HiGHS")` to install the HiGHS package.